In [7]:
import numpy as np
import GeometricTools
import torch
from glob import glob

In [8]:
def wall_x(min_x, max_x, min_y, max_y, min_z, max_z, fix_point):
    
    range_x         = max_x - min_x
    range_y         = max_y - min_y
    range_z         = max_z - min_z

    percent_xz       = range_x / range_z

    num_point_y     = 1
    num_point_x     = int(np.sqrt( fix_point/ num_point_y * percent_xz ))  + 1
    num_point_z     = int(fix_point / num_point_x / num_point_y) 

    # Đây là code để tạo tường random lộn xộn điểm

    # interval_x1 = np.random.uniform(min_x, max_x, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_y1 = np.random.uniform(min_y, thick, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_z1 = np.random.uniform(min_z, max_z, size = (num_point_x, num_point_z, num_point_y) ) 

    #wall_x1 = np.stack((interval_x1, interval_z1, interval_y1), axis = 3).reshape(-1,3)[:fix_point]

    interval_x1 = np.linspace(min_x, max_x, num_point_x) 
    interval_y1 = np.linspace((min_y + max_y)/2, max_y, num_point_y) 
    interval_z1 = np.linspace(min_z, max_z, num_point_z) 

    

    wall = np.array([[a,b,c] for a in interval_x1 for b in interval_y1 for c in interval_z1])[:fix_point]

    return wall

def wall_y(min_x, max_x, min_y, max_y, min_z, max_z, fix_point):

    range_x         = max_x - min_x
    range_y         = max_y - min_y
    range_z         = max_z - min_z

    percent_yz       = range_y / range_z

    num_point_x     = 1
    num_point_y     = int(np.sqrt( fix_point/ num_point_x * percent_yz )) +1
    num_point_z     = int(fix_point / num_point_y / num_point_x) 

    # interval_x1 = np.random.uniform(min_x, thick, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_y1 = np.random.uniform(min_y, max_y, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_z1 = np.random.uniform(min_z, max_z, size = (num_point_x, num_point_z, num_point_y) ) 
    # wall_y1 = np.stack((interval_x1, interval_z1, interval_y1), axis = 3).reshape(-1,3)[:fix_point]

    interval_x1 = np.linspace((min_x+max_x)/2, max_x, num_point_x) 
    interval_y1 = np.linspace(min_y, max_y, num_point_y) 
    interval_z1 = np.linspace(min_z, max_z, num_point_z) 
    


    wall = np.array([[a,b,c] for a in interval_x1 for b in interval_y1 for c in interval_z1])[:fix_point]

    return wall

In [9]:
def downsample_array(original_array, desired_shape):
    # Get the number of rows in the original array
    num_rows_original = original_array.shape[0]
    
    # Generate random indices to select 'b' rows
    random_indices = np.random.choice(num_rows_original, desired_shape[0], replace=False)
    
    # Select rows based on the random indices
    downsampled_array = original_array[random_indices]
    
    return downsampled_array

In [10]:
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.neighbors import NearestNeighbors

# def fps_downsample(original_array, desired_shape):
#     # Calculate pairwise Euclidean distances
#     dist_matrix = euclidean_distances(original_array, original_array)
    
#     # Initialize an array to store selected indices
#     selected_indices = [0]
    
#     # Calculate remaining points using FPS
#     while len(selected_indices) < desired_shape[0]:
#         distances_to_selected = dist_matrix[:, selected_indices]
#         min_distances = np.min(distances_to_selected, axis=1)
#         max_min_distance_index = np.argmax(min_distances)
#         selected_indices.append(max_min_distance_index)
    
#     # Select points based on the selected indices
#     downsampled_array = original_array[selected_indices]
    
#     return downsampled_array

In [11]:
paths = glob('np_points/*')
for path in paths:
    points = np.load(path)
    GeometricTools.drawPointCloudsColorsClasses( torch.tensor(points),  origins=[[0,0,0]]) 
    max_z = np.max(points[:,2])
    projection = points[:,:2] # take only x, y ( n, 2)
    desired_shape = (8000, 2)
    projection = downsample_array(projection, desired_shape)
    new_z = np.linspace(0,max_z, 100)
    
    repeated_array = np.repeat(projection, len(new_z), axis=0)
    repeated_z = np.tile(new_z, len(projection))
    
    new_array = np.column_stack((repeated_array, repeated_z))
    GeometricTools.drawPointCloudsColorsClasses( torch.tensor(new_array),  origins=[[0,0,0]]) 
        

KeyboardInterrupt: 

In [ ]:


# Original array of shape (100, 2)
original_array = np.array([[1,1], [2,2]])

# Array 'z'
z = np.linspace(0,3, 10)

# Repeat the original array for each value in 'z'
repeated_array = np.repeat(original_array, len(z), axis=0)

# Repeat each value in 'z' to match the size of the original array
repeated_z = np.tile(z, len(original_array))

# Combine 'repeated_array' and 'repeated_z' along the second axis
new_array = np.column_stack((repeated_array, repeated_z))

print(new_array)  # Output: (300, 2)


[[1.         1.         0.        ]
 [1.         1.         0.33333333]
 [1.         1.         0.66666667]
 [1.         1.         1.        ]
 [1.         1.         1.33333333]
 [1.         1.         1.66666667]
 [1.         1.         2.        ]
 [1.         1.         2.33333333]
 [1.         1.         2.66666667]
 [1.         1.         3.        ]
 [2.         2.         0.        ]
 [2.         2.         0.33333333]
 [2.         2.         0.66666667]
 [2.         2.         1.        ]
 [2.         2.         1.33333333]
 [2.         2.         1.66666667]
 [2.         2.         2.        ]
 [2.         2.         2.33333333]
 [2.         2.         2.66666667]
 [2.         2.         3.        ]]
